# OSM Python Query

The objective of this notebook is to query OSM database and return list of city and town centroids for all African countries


#### Open street map search 

From Overpass Turbo, use this API and follow the instructions from the
URL below to quary OSM with python

https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0

OSM CRS IS ----  'EPSG:4326'

#### Open Street Map Query for Cities and Villages 
URL for OSM Overpass Turbo: https://overpass-turbo.eu/


URL for OSM Features:
https://wiki.openstreetmap.org/wiki/Map_Features#Populated_settlements.2C_urban

#### Example from Ghana for Towns ... For cities, use *place=city*
    area["name:en"="Ghana"]->.boundaryarea;
    (
    node(area.boundaryarea)[place=town];
    way(area.boundaryarea)[place=town];>;
    rel(area.boundaryarea)[place=town];>>;
    );
    out meta;
    // print results
    out body;
    >;
    out skel qt;

Note: From what I can tell, OSM does not allow for 'Africa' to be used as an 'area' key. 

#### Note: This will pull refugee camps. Really cool. 

In [ ]:
### List of African Countries from the UN in OSM wiki

countries = [
#Northern Africa
    'Algeria',
    'Egypt',
    'Libya',
    'Morocco',
    'Tunisia',
    'Western Sahara',

    #Sub-Saharan Africa
    #Eastern Africa
    'Burundi',
    'Comoros',
    'Djibouti',
    'Eritrea',
    'Ethiopia',
    'Kenya',
    'Madagascar',
    'Malawi',
    'Mauritius',
    #Mayotte,
    'Mozambique',
    'Réunion',
    'Rwanda',
    'Somalia',
    'Sudan',
    'South Sudan',
    'Uganda',
    'Tanzania',
    'Zambia',
    'Zimbabwe',
    
    #Middle Africa
    'Angola',
    'Cameroon',
    'Central African Republic',
    'Chad',
    'Congo-Brazzaville',
    'Democratic Republic of the Congo', # Democratic Republic of the Congo needs to be used, not Congo-Kinshasa
    'Equatorial Guinea',
    'Gabon',
    'Sao Tome and Principe',
    
    # Southern Africa
    'Botswana',
    'Lesotho',
    'Namibia',
    'South Africa',
    'Swaziland',
    
    # Western Africa
    'Benin',
    'Burkina Faso',
    'Cape Verde',
    'Côte d\'Ivoire',
    'Gambia',
    'Ghana',
    'Guinea',
    'Guinea-Bissau',
    'Liberia',
    'Mali',
    'Mauritania',
    'Niger',
    'Nigeria',
    'Senegal',
    'Sierra Leone',
    'Togo'
]

In [ ]:
len(countries)

# Query  OSM Cities and Towns Points for All African Countries

In [ ]:
# import time
import requests
import json

# Check time
# checkpoint = time.time()

# Loop through all OSM countries
response_dict = {}
overpass_url = "http://overpass-api.de/api/interpreter"

# put curly in quotes 

# countries = ['Chad','Ghana', "South Africa"]

for country_name in countries:
    overpass_query = """
            [out:json];
            area["name:en"="{}"];  
            (node[place=city](area);
            );
            out center;
            """.format(country_name)

    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    response_dict[country_name] = data

# print('elasped time to do the quary:{}'.format(time.time() - checkpoint))    

# Cut this on 2019 - 1 - 19 

#            way[place=town](area);
#            rel[place=town](area);

In [ ]:
response_dict

In [ ]:
response_dict['Democratic Republic of the Congo']['elements'][1]['tags']['name']

#### Write out as json file

In [ ]:
# File Paths

IN = ''
OUT = ''

In [ ]:
# Loop through countries and make an np array w/ id, lat, long
import numpy as np
import pandas as pd

def osm_coords(response_dict):
    
    """
    This function takes an OSM dict object and returns a data frame
    with the id, lat, and long ...
    """
    coords = []
    for country in response_dict:    
        for element in response_dict[country]['elements']:
            country = country
            osm_id = str(element['id']) #keep as string for now
            lat = element['lat']
            lon = element['lon']
            
            if 'name' not in element['tags']: 
                city = 'NA'
            else: 
                city = element['tags']['name']
                    
            coords.append((country, osm_id, city, lat, lon,))

    coords_arr = np.array(coords)
    #coords_arr
    
    # make dataframe
    coords_df = pd.DataFrame(data=coords_arr)
    coords_df.columns = ['country','osm_id','town', 'lat','lon']

    return coords_df

In [ ]:
# Run osm_city function

osm_city = osm_coords(response_dict)
osm_city.head()
len(osm_city)

#### Write out as csv w/ only id, lat, long

In [ ]:
# Write out .csv

fn_out = ''
osm_city.to_csv(OUT+fn_out, sep='', index=False, header=True)